# This is a Jupyter Notebook for the Applied Data Science Capstone Project

In [255]:
import pandas as pd
import numpy as np

# Data collection for Takeoutclt.com

## Part 1: Web Scraping from the full Yelp List

In [256]:
from bs4 import BeautifulSoup
import requests

In [257]:
# Yelp webpage - we have the full HTML below, so not needed
#url = "https://www.yelp.com/collection/Z3iwKFIDwScWaYjcBG6v-g?utm_content=Collections&utm_source=ishare"
#url = "https://www.yelp.com/collection/Z3iwKFIDwScWaYjcBG6v-g?sort_by=distance"

In [259]:
#page = requests.get(url)

In [260]:
import os, sys

if sys.version_info.major < 3:
    from urllib import url2pathname
else:
    from urllib.request import url2pathname

class LocalFileAdapter(requests.adapters.BaseAdapter):
    """Protocol Adapter to allow Requests to GET file:// URLs

    @todo: Properly handle non-empty hostname portions.
    """

    @staticmethod
    def _chkpath(method, path):
        """Return an HTTP status for the given filesystem path."""
        if method.lower() in ('put', 'delete'):
            return 501, "Not Implemented"  # TODO
        elif method.lower() not in ('get', 'head'):
            return 405, "Method Not Allowed"
        elif os.path.isdir(path):
            return 400, "Path Not A File"
        elif not os.path.isfile(path):
            return 404, "File Not Found"
        elif not os.access(path, os.R_OK):
            return 403, "Access Denied"
        else:
            return 200, "OK"

    def send(self, req, **kwargs):  # pylint: disable=unused-argument
        """Return the file specified by the given request

        @type req: C{PreparedRequest}
        @todo: Should I bother filling `response.headers` and processing
               If-Modified-Since and friends using `os.stat`?
        """
        path = os.path.normcase(os.path.normpath(url2pathname(req.path_url)))
        response = requests.Response()

        response.status_code, response.reason = self._chkpath(req.method, path)
        if response.status_code == 200 and req.method.lower() != 'head':
            try:
                response.raw = open(path, 'rb')
            except (OSError, IOError) as err:
                response.status_code = 500
                response.reason = str(err)

        if isinstance(req.url, bytes):
            response.url = req.url.decode('utf-8')
        else:
            response.url = req.url

        response.request = req
        response.connection = self

        return response

    def close(self):
        pass

In [491]:
# This gets the actual file
requests_session = requests.session()
requests_session.mount('file://', LocalFileAdapter())
r = requests_session.get('file:///C:/Users/mattw/Downloads/full_clt.html')

In [492]:
soup = BeautifulSoup(r.text)

In [493]:
r_list = soup.findAll(class_ = 'biz-name js-analytics-click')
r2_list = []

In [494]:
for z in range(1,len(r_list)):
    start = str(r_list[z]).find('<span>') + 6
    end = str(r_list[z]).find('</span>')
    name = str(r_list[z])[start:(end-1)]
    #print(name)
    r2_list.append(name)

In [498]:
#r2_list[0:10]

In [549]:
# Empty list
venue_list = []

In [542]:
def get_clt_restaurants(name):
    
    CLIENT_ID = 'XHZAVLMJZPZ1YERWSOQP0RZSG1JTQV24LLJOC3CHJZBFUCWM'
    CLIENT_SECRET = 'FKWRVOLJYEBTMABNFPMRYRJ3JOQLKTDE2JPSZKUGJYHQSAMS'
    VERSION = "20180323"
    
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&query={}'.format(
           CLIENT_ID, 
           CLIENT_SECRET, 
           VERSION, 
           'Charlotte, NC',
           name)
    
    # getting response from query
    resp = requests.get(url=url).json()["response"]['groups'][0]['items'][0]['venue']
    
    # getting website url
    id_url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(
           resp['id'],
           CLIENT_ID, 
           CLIENT_SECRET, 
           VERSION)
    #print(id_url)
    
    
    # Get website name
    venue_info = requests.get(url=id_url).json()['response']['venue']
    
    # Check if there is a website, if not, get delivery - or return nothing
    if 'url' in venue_info:
        website = venue_info['url']
    elif 'delivery' in venue_info:
        website = venue_info['delivery']['url']
    else: website = 'no_url' 
        
    # Check if there is a phone number, if not, get delivery - or return nothing
    if 'formattedPhone' in venue_info['contact']:
        phone_num = venue_info['contact']['formattedPhone']
    else: phone_num = 'no_phone'
        
    # Check if there is a twitter, if not, get delivery - or return nothing
    if 'twitter' in venue_info['contact']:
        twitter = venue_info['contact']['twitter']
    else: twitter = 'no_twitter'   
    
    #print(website)
    
    # return only relevant information for each nearby venue
    print(resp['name'])
    venue_list.append([(resp['name'], 
                    resp['id'],
                   resp['location']['formattedAddress'],
                   resp['location']['lat'],
                   resp['location']['lng'],
                   resp['categories'][0]['shortName'],
                   website,
                   phone_num,
                   twitter)])
    
    return(venue_info)

In [550]:
# loop over restaurants
for ii in r2_list:
    get_clt_restaurants(ii)

Cabo Fish Taco
Hazelnuts Creperie
Fenwicks Restaurant
Three Amigos Mexican Grill and Cantina
TRUE Crafted Pizza
Catawba Brewing Charlotte
The Fillmore Charlotte
The Fillmore Charlotte
Bahn Thai
Viva Chicken Park Road
VBGB Beer Hall & Garden/Restaurant
Chatpatay
Chatpatay
Bánh Mì Brothers
Bánh Mì Brothers
Rita's Italian Ice & Frozen Custard
Benny Pennello's Pizza
Benny Pennello's Pizza
Zen Massage Charlotte
Craft Tasting Room and Growler Shop
Aria Tuscan Grill
Bánh Mì Brothers
Dae Bak Korean Restaurant
Velo Pops
Lego pop-up shop
Angeline's
Stacks Kitchen
Inishmore
Little Village Grill
DD Peckers Wing Shop
Birdsong Brewing Co.
Tony's Ice Cream
VBGB Beer Hall & Garden/Restaurant
Hub City Scoops
Hub City Scoops
Catawba Brewing Charlotte
Veltree Vegan Soul Food And Juice Bar
Viva Chicken Park Road
Viva Chicken Park Road
Viva Chicken Park Road
Viva Chicken Park Road
Bank of America Stadium
Viva Chicken Park Road
Viva Chicken Park Road
Viva Chicken Park Road
Viva Chicken Park Road
Bakersfield

KeyError: 'venue'

In [548]:
venue_list2 = venue_list.copy()

In [558]:
# getting names
names = []
for r in venue_list:
    names.append(r[0][0])

In [559]:
# getting foursquare ID
fid = []
for r in venue_list:
    fid.append(r[0][1])

In [560]:
# getting formatted address
address = []
for r in venue_list:
    address.append(r[0][2][0])

In [561]:
# getting latitude
lat = []
for r in venue_list:
    lat.append(r[0][3])

In [562]:
# getting longitude
lon = []
for r in venue_list:
    lon.append(r[0][4])

In [563]:
# getting categories
cat = []
for r in venue_list:
    cat.append(r[0][5])

In [564]:
# getting website
web = []
for r in venue_list:
    web.append(r[0][6])

In [552]:
# getting phone number
ph_num = []
for r in venue_list:
    ph_num.append(r[0][7])

In [553]:
# getting twitter
twt = []
for r in venue_list:
    twt.append(r[0][8])

In [566]:
venues = pd.DataFrame({'Name':names,
                       'Foursquare_ID':fid, 
                       'Address': address, 
                       'Latitude': lat, 
                       'Longitude':lon, 
                       'Category':cat, 
                       'Website':web,
                       'Phone_Number':ph_num,
                       'Twitter': twt})

In [567]:
venues = venues.drop_duplicates()

In [569]:
venues.to_excel("C:/Users/mattw/Desktop/Charlotte_Restaurants2.xlsx", index = False)  

# CLT Agenda List

In [703]:
cltagn = "https://www.charlotteagenda.com/199599/charlotte-restaurants-that-are-offering-curbside-pickup-delivery-and-discounts/"

In [839]:
# Parsing page from requests
clt_page = requests.get(cltagn)
clt_text = BeautifulSoup(clt_page.text)

In [840]:
ag_list = clt_text.findAll('h2')

In [841]:
websites = []
name = []

for z in range(1,len(ag_list)):
    
    # Getting websites
    web_start = str(ag_list[z]).find('<a href="') + 9
    web_end = str(ag_list[z]).find('" rel')
    web_name = str(ag_list[z])[web_start:(web_end-1)]
    websites.append(web_name)
    
    # Getting Name
    name_start = str(ag_list[z]).find('_blank">') + 8
    name_end = str(ag_list[z]).find('</a>')
    name_restaurant = str(ag_list[z])[name_start:(name_end)]
    name.append(name_restaurant)
    
    print("The restaurant name is {} and the website is {}".format(name_restaurant, web_name))

The restaurant name is Ace No. 3 and the website is https://www.aceno3.com
The restaurant name is Alexander Michael’s and the website is https://almikestavern.com
The restaurant name is Aliño Pizzeria and The Barcelona Burger and Beer Garden and the website is https://barcelonaburger.com
The restaurant name is All-American Pub and the website is https://www.allamericanpubclt.com
The restaurant name is Amelie’s French Cafe &amp; Bakery and the website is https://ameliesfrenchbakery.com
The restaurant name is Anderson’s Catering and the website is https://andersonsdelivery.com
The restaurant name is Angeline’s and the website is https://www.angelinescharlotte.co
The restaurant name is Angry Ale’s and the website is https://www.angryales.com
The restaurant name is Anntony’s Caribbean Cafe and the website is https://www.anntonys.com
The restaurant name is Aria Tuscan Grill and the website is https://www.ariacharlotte.com
The restaurant name is Around The World Cafe and the website is https

In [879]:
clt_agenda = pd.DataFrame({"Name" : name, "Website": websites})

In [880]:
ag_list_par = clt_text.findAll('p')

In [881]:
para = []

for z in range(1,len(ag_list_par)):
    
    # Getting websites
    start = str(ag_list_par[z]).find('<p><em>') + 4
    end = str(ag_list_par[z]).find('</p>')
    para_name = str(ag_list_par[z])[start:(end-1)]
    para.append(para_name)
    
    #print("Paragraph {}: {}".format(z, para_name))

In [882]:
# Legion has three paragraphs, so need to concatenate
para[98] = para[98] + ". " + para[99] + ". " + para[100]

In [883]:
# North Italia
para[133] = para[133] + ". " + para[134]

In [884]:
# OMB
para[137] = para[137] + ". " + para[138] + ". " + para[139]

In [885]:
# Portfolinos
para[148] = para[148] + ". " + para[149] + ". " + para[150] + ". " + para[151] + ". " + para[152]

In [886]:
# Queen City Grounds
para[156] = para[156] + ". " + para[157]

In [887]:
# Deleting copied
to_remove = [99,100,134,138,139,149,150,151,152,157]
para2 = [v for i, v in enumerate(para) if i not in to_remove]

In [888]:
clt_agenda['Information'] = para2[1:-2]

In [889]:
clt_agenda['Information'].replace(regex=True,inplace=True,to_replace="<a href=\".{1,}\">", value="")
clt_agenda['Name'].replace(regex=True,inplace=True,to_replace="href=\".{1,}\">", value="")
clt_agenda['Information'].replace(regex=True,inplace=True,to_replace="</a[>|\s/|.]", value="")
clt_agenda['Information'].replace(regex=True,inplace=True,to_replace="</a$", value="")
clt_agenda['Information'].replace(regex=True,inplace=True,to_replace="amp;", value="")
clt_agenda['Name'].replace(regex=True,inplace=True,to_replace="amp;", value="")
clt_agenda['Name'].replace(regex=True,inplace=True,to_replace="’", value="")
clt_agenda['Name'].replace(regex=True,inplace=True,to_replace="All-American", value="All American")
clt_agenda['Name'].replace(regex=True,inplace=True,to_replace="North Italia", value="North Italia - Charlotte")
clt_agenda['Name'].replace(regex=True,inplace=True,to_replace="OMB", value="The Olde Mecklenburg Brewery")
clt_agenda['Name'].replace(regex=True,inplace=True,to_replace="Pizzeria Omaggio", value="Omaggio Pizzeria")
clt_agenda['Name'].replace(regex=True,inplace=True,to_replace="Crepe Cellar/Growlers Pourhouse/Reigning Donuts", value="Growlers Pourhouse")


In [853]:
# Writing to excel
#clt_agenda.to_excel("C:/Users/mattw/Desktop/Charlotte_Restaurants3.xlsx", index=False)  

### Getting Information from Yelp

In [718]:
# Initializing the API
from yelpapi import YelpAPI
from difflib import get_close_matches 
yelp_api_key = "dkZTgrP07ij03ljhIptABA3WDF613eHyPgDNYwflEJ9q_bCAPMahDc4Fe7zD1c6j1-yj0MEBo7CBgqJg0_w0zdI3tyHobKrrbbb5MJClpeXC_a5QahA2n4wEN-B0XnYx"
yelp_api = YelpAPI(yelp_api_key)

In [890]:
# Adding columns to DF before Yelp search
clt_agenda['longitude'] = ''
clt_agenda['latitude'] = ''
clt_agenda['category'] = ''
clt_agenda['address'] = ''
clt_agenda['display_phone'] = ''
clt_agenda['is_closed'] = ''

In [855]:
def yelp_search(restaurant_name):
    test_search_results = yelp_api.search_query(term= restaurant_name, 
                                                location='charlotte, nc', 
                                                sort_by='best_match',
                                                categories = "restaurants,food,nightlife,catering",
                                                limit=8)
    return(test_search_results)

In [856]:
def close_match_index(patterns, matches):
    
    idx = []
    
    for indx in range(len(patterns)):
        if patterns[indx] in matches:
            idx.append(indx)
            
    return(idx)

In [857]:
def yelp_return_matches(yelp_search, restaurant_n):
    
    # Check for multiple iterations
    like_businesses = []
    
    for rr in yelp_search:
        
        # Checking first word- unless first word is 'The'
        tmp_name = rr['name'].lower().split(' ', 2)
        if tmp_name[0] == 'the':
            tmp_name = tmp_name[1]
        else: tmp_name = tmp_name[0]
        
        like_businesses.append(tmp_name)
        
    # Modifying restaurant name
    tmp_rest = restaurant_n.lower().split(' ', 2)
    if tmp_rest[0] == 'the':
        tmp_rest = tmp_rest[1]
    else:
        tmp_rest = tmp_rest[0]
    
    # Getting similar - only looking at first word of restaurant
    matches = get_close_matches(tmp_rest, like_businesses)
    
    # If one- proceed
    if len(matches) == 1:
        return(yelp_search[0])
    
    else:
        
        # Get index of matches, return indices
        m = close_match_index(like_businesses, matches)
    
        multiple_location_index = []
        for loc_index in m:
            multiple_location_index.append(yelp_search[loc_index])
        return(multiple_location_index)

In [895]:
# Loop over all restaurants
for ii in range(len(clt_agenda['Name'])):
    
    # Restaurant Name
    n = clt_agenda.iloc[ii,0]
    print('We are working on {}'.format(n))
    
    # Get Yelp Search
    yelp_return = yelp_search(n)
    yelp_return = yelp_return['businesses']
    
    # Check for doubles
    restaurant_info = yelp_return_matches(yelp_return, n)
    print(len(restaurant_info))#restaurant_info
    
    # If we can proceed, add to dataframe
    if len(restaurant_info) == 0:
        pass
    elif len(restaurant_info) > 10: # if there is one return, length defaults to 1 dictionary with 15 entries (maybe
                                  # more, so just putting a number > max entries returned)
        
        clt_agenda.at[ii, 'longitude'] = restaurant_info['coordinates']['longitude']
        clt_agenda.at[ii, 'latitude'] = restaurant_info['coordinates']['latitude']
        clt_agenda.at[ii, 'category'] = restaurant_info['categories'][0]['title']
        clt_agenda.at[ii, 'address'] = restaurant_info['location']['display_address']
        clt_agenda.at[ii, 'display_phone'] = restaurant_info['display_phone']
        clt_agenda.at[ii, 'is_closed'] = restaurant_info['is_closed']
        
    else:
        
        lng = []
        lati = []
        addy = []
        phn = []
        isclosed = []
        
        for mult in range(len(restaurant_info)):
            
            lng.append(restaurant_info[mult]['coordinates']['longitude'])
            lati.append(restaurant_info[mult]['coordinates']['latitude'])
            addy.append(restaurant_info[mult]['location']['display_address'])
            phn.append(restaurant_info[mult]['display_phone'])            
            isclosed.append(restaurant_info[mult]['is_closed'])            
   
        clt_agenda.at[ii, 'longitude'] = lng
        clt_agenda.at[ii, 'latitude'] = lati
        clt_agenda.at[ii, 'category'] = restaurant_info[0]['categories'][0]['title']
        clt_agenda.at[ii, 'address'] = addy
        clt_agenda.at[ii, 'display_phone'] = phn  
        clt_agenda.at[ii, 'is_closed'] = isclosed
        

We are working on Ace No. 3
15
We are working on Alexander Michaels
16
We are working on Aliño Pizzeria and The Barcelona Burger and Beer Garden
16
We are working on All American Pub
16
We are working on Amelies French Cafe & Bakery
4
We are working on Andersons Catering
15
We are working on Angelines
16
We are working on Angry Ales
16
We are working on Anntonys Caribbean Cafe
2
We are working on Aria Tuscan Grill
16
We are working on Around The World Cafe
2
We are working on The Asbury
16
We are working on Bad Daddys
6
We are working on Bahn Mi & Tea Company
16
We are working on Bahn Thai
16
We are working on Bardo
16
We are working on Bar Marcel
16
We are working on Barringtons
16
We are working on Barristers
15
We are working on Bedder Bedder & More
16
We are working on Be-Em Asian Kitchen
16
We are working on Blaze Pizza
3
We are working on Blue Blaze Brewing
16
We are working on Boardwalk Billys
2
We are working on Bojangles
8
We are working on Bonzai Thai & Japanese Cuisine
16
We

In [896]:
clt_agenda2 = clt_agenda.loc[(clt_agenda.is_closed != True)].drop('is_closed', axis = 1)

In [897]:
clt_agenda2.to_excel("C:/Users/mattw/Desktop/Charlotte_Restaurants5.xlsx", index=False)  

In [655]:
# Writing to excel
#xxx.to_excel("C:/Users/mattw/Desktop/Charlotte_Restaurants4.xlsx", index=False)  

In [899]:
xxx2 = clt_agenda2.copy()

In [900]:
rows_to_delete = []

for row in range(len(clt_agenda2)):
    
    # Restaurant Name, website, information, category
    rest_n = clt_agenda2.iloc[row, 0]
    rest_web = clt_agenda2.iloc[row, 1]
    rest_info = clt_agenda2.iloc[row, 2]
    rest_cat = clt_agenda2.iloc[row, 5]
    
    # Check if there are multiples - it will return a list vs. float
    if type(clt_agenda2.iloc[row, 3]) is list:
        
        # If empty- will return 0, so just removing those
        if len(clt_agenda2.iloc[row, 3]) > 0:
            
            # Append to drop column list for housekeeping later
            rows_to_delete.append(row)
            
            # Looping over entries
            for location in range(len(clt_agenda2.iloc[row,3])):
                
                # Getting restaurant specific information
                specific_longitude = clt_agenda2.iloc[row,3][location]
                specific_latitude = clt_agenda2.iloc[row, 4][location]
                specific_address = clt_agenda2.iloc[row, 6][location]
                specific_phone = clt_agenda2.iloc[row, 7][location]
                
                new_row = {'Name': rest_n, 
                           'Website': rest_web, 
                           'Information': rest_info, 
                           'longitude': specific_longitude, 
                           'latitude': specific_latitude,
                           'category':rest_cat,
                           'address':specific_address,
                           'display_phone':specific_phone}
                
                #append row to the dataframe
                xxx2 = xxx2.append(new_row, ignore_index=True)

xxx2 = xxx2.drop(xxx2.index[rows_to_delete])

In [901]:
xxx2.to_excel("C:/Users/mattw/Desktop/Charlotte_Restaurants6.xlsx", index=False)  

# Map for Charlotte Restaurants

In [904]:
import folium
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors

In [941]:
map_clt = xxx2.copy()
map_clt = map_clt[map_clt['latitude'] != '']

In [942]:
map_clt['longitude'] = map_clt['longitude'].apply(float)
map_clt['latitude'] = map_clt['latitude'].apply(float)
map_clt['Name'] = map_clt['Name'].apply(str)
map_clt['category'] = map_clt['category'].apply(str)

In [922]:
categories = map_clt['category'].unique()
colors_array = cm.rainbow(np.linspace(0, 1, len(categories)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [925]:
categories

array(['Burgers', 'American (Traditional)', 'American (New)', 'Caterers',
       'Italian', 'Sports Bars', 'Southern', 'Vietnamese', 'Thai',
       'Wine Bars', 'Breakfast & Brunch', 'Japanese', 'Breweries',
       'Chicken Shop', 'Brewpubs', 'Tapas Bars', 'Bars', 'Beer Gardens',
       'Health Markets', 'Coffee & Tea', 'Sushi Bars', 'Beer Bar',
       'Steakhouses', 'Grocery', 'Sandwiches', 'Ramen', 'Asian Fusion',
       'Seafood', 'Beer, Wine & Spirits', 'Mexican', 'Mediterranean',
       'Public Markets', 'Vegetarian', 'Bakeries', 'Pizza', 'Barbeque',
       'French', 'Diners', 'Venues & Event Spaces', 'Food Court',
       'Chinese', 'Vegan', 'Pubs', 'Poke', 'Tex-Mex', 'Tacos',
       'Chocolatiers & Shops', 'Soul Food', 'Pasta Shops', 'Cafes',
       'Caribbean', 'Ice Cream & Frozen Yogurt', 'Salad', 'Fast Food',
       'Indian', 'Chicken Wings', 'Delis', 'Donuts', 'Greek',
       'Juice Bars & Smoothies', 'Hot Dogs', 'African', 'Latin American',
       'Desserts', 'Peruvian', 'Mi

In [482]:
colors = [
    'red',
    'blue',
    'gray',
    'darkred',
    'lightred',
    'orange',
    'beige',
    'green',
    'darkgreen',
    'lightgreen',
    'darkblue',
    'lightblue',
    'purple',
    'darkpurple',
    'pink',
    'cadetblue',
    'lightgray',
    'black'
]

In [943]:
# create map of New York using latitude and longitude values
map_clt_new = folium.Map(location=[35.228537,-80.8441127], zoom_start=12)

for lat, lon, n, cat  in zip(map_clt['latitude'], map_clt['longitude'], map_clt['Name'], map_clt['category']):
    
    # Getting Category
    tmp_cat = np.where(categories == cat)[0][0]
    tmp_color = rainbow[tmp_cat]
    
    label = folium.Popup("<b>Name: </b>" + str(n)  + "<br> <b>Category:</b> " + str(cat) + '<br><a href="http://www.google.com">Google</a>')
    folium.Marker(
        [lat, lon],
        #radius=7,
        popup=label,
        icon = folium.Icon(color = 'blue')).add_to(map_clt_new)
        #fill=True,
        #fill_color=tmp_color,
        #fill_opacity=0.7).add_to(map_clt)
map_clt_new

In [944]:
map_clt_new.save('C:/Users/mattw/Desktop/clt_map_initial2.html')